In [ ]:
import os
import datetime
import google.auth
from googleapiclient.discovery import build

# --- CONFIGURACIÓN ---
CREDENTIALS_FILE = 'google_credentials.json' 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = CREDENTIALS_FILE

def get_service():
    """Conecta con la API de Google Calendar"""
    if not os.path.exists(CREDENTIALS_FILE):
        raise FileNotFoundError(f"No encuentro el archivo {CREDENTIALS_FILE}. Verifica la ruta.")
        
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    creds, _ = google.auth.default(scopes=SCOPES)
    service = build('calendar', 'v3', credentials=creds)
    print("✅ Autenticación exitosa con Google Calendar API")
    return service

# Probamos la conexión
service = get_service()

In [ ]:
now = datetime.datetime.utcnow().isoformat() + 'Z' 
now

In [ ]:
def listar_eventos(service, max_results=5):
    """Lista los próximos eventos en el calendario principal"""
    # Obtenemos la hora actual en formato ISO UTC
    now = datetime.datetime.utcnow().isoformat() + 'Z' 
    
    print(f"🔍 Buscando los próximos {max_results} eventos a partir de ahora...")
    
    events_result = service.events().list(
        calendarId='delfosgz@gmail.com', 
        timeMin=now,
        maxResults=max_results, 
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    
    events = events_result.get('items', [])

    if not events:
        print("No se encontraron eventos próximos.")
        return

    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        summary = event.get('summary', '(Sin título)')
        print(f"📅 {start} | {summary}")

# Ejecutar prueba de lectura
listar_eventos(service)

In [ ]:
service

In [ ]:
from zoneinfo import ZoneInfo # <--- Importante
import datetime

def get_calendar_events(day: str = None):
    
    # 1. Definimos la zona horaria explícitamente
    zona_cdmx = ZoneInfo("America/Mexico_City")
    
    if not day:
        # Usamos 'now' CON zona horaria para evitar desfases de día
        day = datetime.datetime.now(zona_cdmx).strftime("%Y-%m-%d")

    # 2. Convertimos el string 'YYYY-MM-DD' a objetos datetime conscientes de la zona
    # Esto asegura que start sea 00:00 CDMX y end sea 23:59 CDMX
    fecha_dt = datetime.datetime.strptime(day, "%Y-%m-%d")
    
    start_dt = fecha_dt.replace(hour=0, minute=0, second=0, tzinfo=zona_cdmx)
    end_dt = fecha_dt.replace(hour=23, minute=59, second=59, tzinfo=zona_cdmx)
    
    # 3. Convertimos a ISO format (esto pondrá automáticamente el -06:00 en vez de la Z)
    start_time = start_dt.isoformat()
    end_time = end_dt.isoformat()
    
    print(f"--- RANGO DE BÚSQUEDA REAL (ISO): {start_time} a {end_time} ---")
    
    try:
        events_result = service.events().list(
            calendarId='delfosgz@gmail.com',  # Tu calendario personal
            timeMin=start_time, 
            timeMax=end_time,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            return f"No hay eventos programados para el {day}."
        
        result = f"Eventos para {day}:\n"
        for event in events:
            # Extraemos fecha/hora y limpiamos formato
            raw_start = event['start'].get('dateTime', event['start'].get('date'))
            summary = event.get('summary', 'Sin título')
            result += f"- {raw_start}: {summary}\n"
            
        return result

    except Exception as e:
        return f"Error en API: {e}"

In [ ]:
get_calendar_events('2026-01-30')

In [ ]:
import datetime
from zoneinfo import ZoneInfo 

def crear_evento_prueba(service):
    """Crea un evento para MAÑANA a la MISMA HORA que es ahorita"""
    
    # 1. Obtenemos hora actual exacta en CDMX
    zona_cdmx = ZoneInfo("America/Mexico_City")
    now_cdmx = datetime.datetime.now(zona_cdmx)
    
    print(f"1. Hora actual (CDMX): {now_cdmx}")

    # 2. Calculamos MAÑANA A ESTA HORA
    # Simplemente sumamos 1 día al reloj actual
    tomorrow_same_time = now_cdmx + datetime.timedelta(days=1)
    
    # 3. LIMPIEZA DE HORA (El paso clave)
    # Extraemos los números puros (Año, Mes, Día, Hora, Minuto) de la variable calculada
    # y creamos un objeto nuevo SIN zona horaria para que no lleve el "-06:00"
    start_time_clean = datetime.datetime(
        tomorrow_same_time.year, 
        tomorrow_same_time.month, 
        tomorrow_same_time.day, 
        tomorrow_same_time.hour,   # <--- Aquí toma la hora actual (ej: 16)
        tomorrow_same_time.minute, # <--- Aquí toma el minuto actual (ej: 53)
        tomorrow_same_time.second
    )
    
    end_time_clean = tomorrow_same_time + datetime.timedelta(hours=1)
    
    # 4. Formateamos a ISO
    start_str = tomorrow_same_time.isoformat()
    end_str = end_time_clean.isoformat()

    print(f"2. Mandando a Google: {start_str} (en zona America/Mexico_City)")

    event_body = {
        'summary': '🤖 Test Dinámico (Mañana misma hora)',
        'location': 'Docker Container',
        'description': 'Este evento usa la hora dinámica de ejecución + 1 día.',
        'start': {
            'dateTime': start_str,             
            'timeZone': 'America/Mexico_City', 
        },
        'end': {
            'dateTime': end_str,
            'timeZone': 'America/Mexico_City',
        },
    }

    print("✍️ Creando evento...")
    
    try:
        event = service.events().insert(calendarId='delfosgz@gmail.com', body=event_body).execute()
        print(f"✅ Evento creado: {event.get('htmlLink')}")
    except Exception as e:
        print(f"❌ Error: {e}")

# Ejecutar
crear_evento_prueba(service)